<a href="https://colab.research.google.com/github/MiracleCakra/Machine-Learning_Ganjil_2025/blob/main/Week07_JS07/Praktikum03_JS07_HNSWLIB_Hierarchical_Navigable_Small_World_graphs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Praktikum 3

HNSW

In [3]:
!pip install hnswlib -q

Percobaan berikut akan membandingkan exact NN dengan HNSW pada 1000 data 2D.

In [4]:
import hnswlib
import numpy as np
import time
from sklearn.neighbors import NearestNeighbors

# ===========================
# 1. Buat data 2D acak
# ===========================
num_elements = 1000
dim = 2
data = np.random.random((num_elements, dim)).astype(np.float32)

# Query point
query = np.array([[0.5, 0.5]], dtype=np.float32)
k = 5  # cari 5 tetangga terdekat

# ===========================
# 2. Exact NN (Brute Force)
# ===========================
nn = NearestNeighbors(n_neighbors=k, algorithm='brute', metric='euclidean')
nn.fit(data)

start = time.time()
distances, indices = nn.kneighbors(query)
end = time.time()

print("=== Exact NN ===")
print("Indices:", indices)
print("Distances:", distances)
print("Waktu:", end - start, "detik")

# ===========================
# 3. HNSW
# ===========================
# Inisialisasi index HNSW
p = hnswlib.Index(space='l2', dim=dim)

# Ukuran maksimum elemen yang bisa ditampung
p.init_index(max_elements=num_elements, ef_construction=100, M=16)

# Tambahkan data
p.add_items(data)

# Set parameter pencarian
p.set_ef(50)   # tradeoff speed vs accuracy

start = time.time()
labels, distances = p.knn_query(query, k=k)
end = time.time()

print("\n=== HNSW ===")
print("Indices:", labels)
print("Distances:", distances)
print("Waktu:", end - start, "detik")


=== Exact NN ===
Indices: [[905 570 606  14  68]]
Distances: [[0.02183795 0.03808458 0.04583036 0.05158029 0.05793338]]
Waktu: 0.18595576286315918 detik

=== HNSW ===
Indices: [[905 570 606  14  68]]
Distances: [[0.0004769  0.00145044 0.00210042 0.00266053 0.00335628]]
Waktu: 0.00023865699768066406 detik


## Lakukan percobaan pada metric distance yang berbeda, 1000 vs 1jt data, 2D vs 5D data. catat hasilnya pada tabel yang anda buat sendiri seperti pada praktikum 1.

In [5]:
import hnswlib
import numpy as np
import time
import pandas as pd
from sklearn.neighbors import NearestNeighbors

def avg_search_time(callable_fn, repeats=3):
    times = []
    for _ in range(repeats):
        t0 = time.time()
        callable_fn()
        times.append(time.time() - t0)
    return float(np.mean(times))

def run_experiment(n_data, dim, metric_type, repeats=3,
                   M=16, ef_construction=200, ef=50):
    """
    HNSW (approx) vs Brute-force (exact) untuk:
    - n_data: 1_000 dan 1_000_000
    - dim: 2 dan 5
    - metric_type: 'l2' atau 'ip' (IP + normalisasi ≈ Cosine)
    """
    print(f"Running experiment: {n_data} data, {dim}D, Metric: {metric_type}...")

    np.random.seed(42)
    data = np.random.random((n_data, dim)).astype(np.float32)
    query = np.random.random((1, dim)).astype(np.float32)
    k = 5

    # Normalisasi jika IP (agar setara Cosine)
    if metric_type == 'ip':
        eps = 1e-12
        data_norm = np.linalg.norm(data, axis=1, keepdims=True) + eps
        data = data / data_norm
        query = query / (np.linalg.norm(query, axis=1, keepdims=True) + eps)
        sklearn_metric = 'cosine'
    else:
        sklearn_metric = 'euclidean'

    # ---- Exact (Sklearn brute) ----
    nn = NearestNeighbors(n_neighbors=k, algorithm='brute', metric=sklearn_metric)
    nn.fit(data)

    # 1x untuk indeks ground-truth
    t0 = time.time()
    _, indices_exact = nn.kneighbors(query)
    time_exact_once = time.time() - t0

    # rata-rata waktu search exact
    time_exact = avg_search_time(lambda: nn.kneighbors(query), repeats=repeats)

    # ---- HNSW (Approx) ----
    p = hnswlib.Index(space=metric_type, dim=dim)
    t_build0 = time.time()
    p.init_index(max_elements=n_data, ef_construction=ef_construction, M=M)
    p.add_items(data)
    time_build = time.time() - t_build0

    p.set_ef(ef)

    # rata-rata waktu search HNSW
    def _hnsw_query():
        p.knn_query(query, k=k)
    time_hnsw = avg_search_time(_hnsw_query, repeats=repeats)

    # ambil 1 hasil untuk hitung recall
    indices_hnsw, _ = p.knn_query(query, k=k)

    true_neighbors = set(indices_exact[0])
    approx_neighbors = set(indices_hnsw[0])
    recall = len(true_neighbors.intersection(approx_neighbors)) / k

    speedup = (time_exact / time_hnsw) if time_hnsw > 0 else None

    return {
        "Ukuran Data": n_data,
        "Dimensi": dim,
        "Metrik Jarak": metric_type.upper(),
        "M (HNSW)": M,
        "ef_construction": ef_construction,
        "ef (search)": ef,
        "Waktu Build (HNSW) (s)": round(time_build, 6),
        "Waktu Search (Exact) (s)": round(time_exact, 6),
        "Waktu Search (HNSW) (s)": round(time_hnsw, 6),
        f"Recall@{k}": round(recall, 4),
        "Speedup (Exact/HNSW)": round(speedup, 2) if speedup is not None else None
    }

# --- Eksekusi Semua Skenario ---
scenarios = [
    (1000, 2, 'l2'),
    (1000, 5, 'l2'),
    (1000000, 2, 'l2'),
    (1000000, 5, 'l2'),
    (1000, 2, 'ip'),
    (1000, 5, 'ip'),
    (1000000, 2, 'ip'),
    (1000000, 5, 'ip'),
]

results = []
for n_data, dim, metric in scenarios:
    try:
        results.append(run_experiment(n_data, dim, metric, repeats=3,
                                      M=16, ef_construction=200, ef=50))
    except MemoryError:
        # Jika RAM tidak cukup untuk 1 juta, tetap catat barisnya
        results.append({
            "Ukuran Data": n_data, "Dimensi": dim, "Metrik Jarak": metric.upper(),
            "M (HNSW)": 16, "ef_construction": 200, "ef (search)": 50,
            "Waktu Build (HNSW) (s)": None,
            "Waktu Search (Exact) (s)": None,
            "Waktu Search (HNSW) (s)": None,
            "Recall@5": None,
            "Speedup (Exact/HNSW)": None
        })

df_results = pd.DataFrame(results)

df_results = df_results.sort_values(by=["Ukuran Data","Dimensi","Metrik Jarak"]).reset_index(drop=True)

print("\n--- Hasil Eksperimen ---")
print(df_results.to_string(index=False))

# Konversi ke Markdown
md_table = df_results.to_markdown(index=False)
print("\n--- Markdown Table (buat laporan) ---")
print(md_table)

Running experiment: 1000 data, 2D, Metric: l2...
Running experiment: 1000 data, 5D, Metric: l2...
Running experiment: 1000000 data, 2D, Metric: l2...
Running experiment: 1000000 data, 5D, Metric: l2...
Running experiment: 1000 data, 2D, Metric: ip...
Running experiment: 1000 data, 5D, Metric: ip...
Running experiment: 1000000 data, 2D, Metric: ip...
Running experiment: 1000000 data, 5D, Metric: ip...

--- Hasil Eksperimen ---
 Ukuran Data  Dimensi Metrik Jarak  M (HNSW)  ef_construction  ef (search)  Waktu Build (HNSW) (s)  Waktu Search (Exact) (s)  Waktu Search (HNSW) (s)  Recall@5  Speedup (Exact/HNSW)
        1000        2           IP        16              200           50                0.055053                  0.001293                 0.000038       1.0                 33.68
        1000        2           L2        16              200           50                0.128308                  0.000655                 0.000039       1.0                 16.85
        1000        5   

| Ukuran Data | Dimensi | Metrik Jarak | Waktu Build (HNSW) (s) | Waktu Search (Exact) (s) | Waktu Search (HNSW) (s) | Recall@5 |
| :---------: | :-----: | :----------: | :--------------------: | :----------------------: | :---------------------: | :------: |
|    1000     |    2    |      L2      |        0.009001        |         0.002996         |        0.000000         |   1.0    |
|    1000     |    5    |      L2      |        0.007998        |         0.004562         |        0.000000         |   1.0    |
|   1000000   |    2    |      L2      |       12.098200        |         0.006001         |        0.000000         |   1.0    |
|   1000000   |    5    |      L2      |       18.497900        |         0.009999         |        0.000000         |   1.0    |
|    1000     |    2    |      IP      |        0.007508        |         0.000516         |        0.000000         |   1.0    |
|    1000     |    5    |      IP      |        0.008002        |         0.000999         |        0.000000         |   1.0    |
|   1000000   |    2    |      IP      |       18.336600        |         0.027482         |        0.000000         |   0.0    |
|   1000000   |    5    |      IP      |       16.405500        |         0.036579         |        0.000000         |   1.0    |
